In [1]:
# setup our environment

import pandas as pd
import numpy as np

DATA_FILE = "data/fake_or_real_news.csv"
TEST_SIZE = 0.2

In [2]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)
    
    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)
    
    return train_acc, test_acc

In [3]:
# estimate 95% confidence interval on error

# NOTE: based on conversation on stackexchange: 
# https://stats.stackexchange.com/questions/247551/how-to-determine-the-confidence-of-a-neural-network-prediction
# towards bottom of the page.

from math import sqrt

def error_conf(error, n):
    term = 1.96*sqrt((error*(1-error))/n)
    lb = error - term
    ub = error + term
    
    return lb, ub

In [4]:
# read in our data and preprocess it
# Note: the news items in the data set range from 0 words to 100,000 words
# we restrict ourselves to news items between 500 and 5,000 words.

df = pd.read_csv(DATA_FILE)
df.drop(labels=['id','title'], axis='columns', inplace=True)
# only select stories with between 500 and 5000 words
mask = list(df['text'].apply(lambda x: len(x) >= 500 and len(x) <= 5000))
df = df[mask]

In [5]:
print(df.shape)

(3510, 2)


In [6]:
# show that the sample data is pretty balanced
print(len(df[df['label'] == 'REAL']), len(df[df['label'] == 'FAKE']))

1472 2038


In [7]:
X = df['text']
y = df['label']

In [8]:
# convert labels to numeric labels
# NOTE: DNNs need numeric labels

def convert(x):
    if x == 'FAKE':
        return 0.0
    else:
        return 1.0


## Syntactic Features

In [9]:
# set up vector models for training and testing

from sklearn.feature_extraction.text import CountVectorizer

# data vectorizer
vectorizer = CountVectorizer(analyzer = "word", 
                             binary = True, 
                             min_df = 2,
                             stop_words='english')
docarray = vectorizer.fit_transform(X).toarray()
docterm = pd.DataFrame(docarray, columns=vectorizer.get_feature_names())

In [10]:
print(docterm.shape)

(3510, 23516)


In [11]:
# create training and test data
from sklearn.model_selection import train_test_split
docterm_train, docterm_test, y_train, y_test = train_test_split(docterm, y, test_size=TEST_SIZE)

### Naive Bayes

In [12]:
# Naive Bayes Model - train & test
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(docterm_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, docterm_train, y_train, docterm_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 96.44%
Testing Accuracy: 87.46%


In [14]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 85.01%-89.91%


### Random Forest

In [15]:
# Random Forest Model - train & test
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(min_samples_split=60)
model.fit(docterm_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=60,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, docterm_train, y_train, docterm_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 95.44%
Testing Accuracy: 83.48%


In [17]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 80.73%-86.22%


### Deep Neural Network

In [18]:
# DNN
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# build the model
model = Sequential()
model.add(Dense(1024, input_dim=docterm_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(docterm_train, y_train.apply(convert),
          epochs=20,
          batch_size=128,
          validation_data=(docterm_test, y_test.apply(convert)))

Using TensorFlow backend.


Train on 2808 samples, validate on 702 samples
Epoch 1/20
2808/2808 [==============================] - 23s 8ms/step - loss: 0.6071 - acc: 0.6560 - val_loss: 0.3909 - val_acc: 0.8048
Epoch 2/20
2808/2808 [==============================] - 19s 7ms/step - loss: 0.2406 - acc: 0.9135 - val_loss: 0.1736 - val_acc: 0.9274
Epoch 3/20
2808/2808 [==============================] - 22s 8ms/step - loss: 0.0465 - acc: 0.9882 - val_loss: 0.2290 - val_acc: 0.9330
Epoch 4/20
2808/2808 [==============================] - 20s 7ms/step - loss: 0.0153 - acc: 0.9961 - val_loss: 0.3635 - val_acc: 0.9288
Epoch 5/20
2808/2808 [==============================] - 20s 7ms/step - loss: 0.0016 - acc: 0.9996 - val_loss: 0.4920 - val_acc: 0.9302
Epoch 6/20
2808/2808 [==============================] - 20s 7ms/step - loss: 0.0022 - acc: 0.9993 - val_loss: 0.6523 - val_acc: 0.9302
Epoch 7/20
2808/2808 [==============================] - 22s 8ms/step - loss: 3.7626e-04 - acc: 1.0000 - val_loss: 0.7033 - val_acc: 0.9274
Epoc

In [19]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict_classes, 
                                     docterm_train, 
                                     y_train.apply(convert), 
                                     docterm_test, 
                                     y_test.apply(convert))
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 100.00%
Testing Accuracy: 92.17%


In [20]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 90.18%-94.15%


## Semantic Features

In [21]:
# Load Spacy semantic model

import spacy

EMBEDDING_DIM = 300

# NOTE: for performance reasons disable everything in the pipeline except the tokenizer
nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner', 'textcat'])

In [22]:
# preprocess text for semantic features

def embed(X):
    '''
    x is a list of strings and embed will compute
    an embedding vector for each and return an array
    of shape (len(x),EMBEDDING_DIM)
    '''
    vectors = []
    text_array = np.array(X)

    print(text_array.shape)
    
    for i in range(text_array.shape[0]):
        vector = nlp(text_array[i]).vector
        vectors.append(vector)
    
    return pd.DataFrame(vectors)

vectors = embed(X)

(3510,)


In [23]:
# create training and test data
from sklearn.model_selection import train_test_split
vectors_train, vectors_test, y_train, y_test = train_test_split(vectors, y, test_size=TEST_SIZE)

### Naive Bayes

In [24]:
# Naive Bayes Model 
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(vectors_train,y_train)

GaussianNB(priors=None)

In [25]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, vectors_train, y_train, vectors_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 73.79%
Testing Accuracy: 70.66%


In [26]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 67.29%-74.02%


### Random Forest

In [27]:
# Random Forest Model 
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(min_samples_split=10)
model.fit(vectors_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict, vectors_train, y_train, vectors_test, y_test)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 98.79%
Testing Accuracy: 81.20%


In [29]:
# computing 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 78.31%-84.09%


### Deep Neural Network

In [30]:
# DNN - MLP
from keras.models import Sequential
from keras.layers import Dense, Dropout

# build the model
model = Sequential()
model.add(Dense(256, input_dim=EMBEDDING_DIM, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [31]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(vectors_train, y_train.apply(convert),
          epochs=15,
          batch_size=128,
          validation_data=(vectors_test, y_test.apply(convert)))

Train on 2808 samples, validate on 702 samples
Epoch 1/15
2808/2808 [==============================] - 4s 2ms/step - loss: 0.6814 - acc: 0.5666 - val_loss: 0.6571 - val_acc: 0.5698
Epoch 2/15
2808/2808 [==============================] - 1s 307us/step - loss: 0.6331 - acc: 0.6296 - val_loss: 0.5504 - val_acc: 0.7892
Epoch 3/15
2808/2808 [==============================] - 1s 314us/step - loss: 0.5412 - acc: 0.7382 - val_loss: 0.4887 - val_acc: 0.7863
Epoch 4/15
2808/2808 [==============================] - 1s 327us/step - loss: 0.4655 - acc: 0.7970 - val_loss: 0.4320 - val_acc: 0.8276
Epoch 5/15
2808/2808 [==============================] - 1s 320us/step - loss: 0.4151 - acc: 0.8162 - val_loss: 0.3791 - val_acc: 0.8333
Epoch 6/15
2808/2808 [==============================] - 1s 335us/step - loss: 0.3893 - acc: 0.8401 - val_loss: 0.3704 - val_acc: 0.8376
Epoch 7/15
2808/2808 [==============================] - 1s 334us/step - loss: 0.3578 - acc: 0.8515 - val_loss: 0.3403 - val_acc: 0.8632
Epo

In [32]:
# evaluate model
train_acc, test_acc = evaluate_model(model.predict_classes, 
                                     vectors_train, 
                                     y_train.apply(convert), 
                                     vectors_test, 
                                     y_test.apply(convert))
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 91.81%
Testing Accuracy: 88.46%


In [33]:
# estimating 95% confidence interval
n = docterm_test.shape[0]
lb, ub = error_conf(1-test_acc, n)

print("95% confidence interval: {:.2f}%-{:.2f}%".format((1-ub)*100,(1-lb)*100))

95% confidence interval: 86.10%-90.82%
